In [1]:
import numpy as np
from collections import namedtuple
import cv2
from pathlib import Path
import argparse
import os
from openvino.inference_engine import IENetwork, IECore
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow import keras

c:\Users\de31\anaconda3\envs\lp\lib\site-packages\tensorflow\python\framework\dtypes.py:205: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
c:\Users\de31\anaconda3\envs\lp\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


## 전처리 먼저

In [2]:
folder_path_1 = "./data/구매(구매)"
folder_path_2 = "./data/구매(반품)"
folder_path_3 = "./data/구매(비교)"
folder_path_4 = "./data/구매(선택)"
folder_path_5 = "./data/구매(시험)"

In [25]:
def concat(path):
    file_lst = os.listdir(path)
    df_list = []
    for file in file_lst:
        file_name = os.path.join(path, file)
        df = pd.read_csv(file_name)
        df = df[df.NUMOFBODIES != 0].reset_index(drop=True)
        df_list.append(df)
    #concat_df = pd.concat(df_list, ignore_index=True)
    return df_list

In [26]:
buy_mer = concat(folder_path_1)
refund_mer = concat(folder_path_2)
compar_mer = concat(folder_path_3)
select_mer = concat(folder_path_4)
test_mer = concat(folder_path_5)

In [28]:
buy_mer[0]

,NUMOFBODIES,ID,PELVIS_X,PELVIS_Y,PELVIS_CONFIDENCE_LEVEL,SPINE_NAVAL_X,SPINE_NAVAL_Y,SPINE_NAVA_CONFIDENCE_LEVEL,SPINE_CHEST_X,SPINE_CHEST_Y,...,KNEE_RIGHT_Y,KNEE_RIGHT_CONFIDENCE_LEVEL,ANKLE_RIGHT_X,ANKLE_RIGHT_Y,ANKLE_RIGHT_CONFIDENCE_LEVEL,HEAD_X,HEAD_Y,HEAD_CONFIDENCE_LEVEL,TIMESTAMP,FRAME_NUM
0,1,0,346.55383,640.09490,2,316.11750,576.43010,2,280.24127,522.70620,...,752.30030,2,434.75787,857.02515,2,239.81610,388.57196,2,14:46:38:326,63
1,1,0,374.96075,649.16760,2,356.46484,581.53500,2,337.61346,522.35820,...,813.82970,2,387.14087,889.73960,2,265.55145,386.76953,2,14:46:38:435,66
2,1,0,429.70477,641.58417,2,411.60730,574.76430,2,388.89648,514.81540,...,818.55020,2,392.70930,887.81946,2,331.37190,371.98608,2,14:46:38:518,69
3,1,0,495.43500,632.74255,2,477.77640,565.54290,2,457.34350,506.20636,...,806.14215,2,410.20886,890.77374,2,402.76740,364.13617,2,14:46:38:630,72
4,1,0,547.60315,616.55760,2,527.22210,550.83923,2,507.37710,493.02690,...,790.46180,2,479.45310,865.62000,2,463.35953,355.15625,2,14:46:38:726,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,1,0,441.13635,513.26570,2,418.80603,447.77664,2,397.13898,390.69366,...,617.61170,2,545.41315,725.03420,2,318.51190,259.53024,2,14:47:24:246,1440
460,1,0,410.09950,518.18020,2,383.44873,454.08856,2,362.42060,399.03620,...,620.71770,2,540.12720,717.93620,2,282.22357,273.52228,1,14:47:24:344,1443
461,1,0,379.14062,521.87560,2,346.83826,461.59387,2,321.31458,409.53815,...,618.06915,2,540.31460,710.00745,2,244.51929,288.37540,1,14:47:24:438,1446
462,1,0,329.50354,550.63586,2,305.44708,497.32147,2,287.64514,452.11716,...,622.12260,2,511.84430,699.06330,2,225.02637,351.35684,1,14:47:24:546,1449


In [5]:
buy_mer['LABEL'] = 0  
refund_mer['LABEL'] = 1  
compar_mer['LABEL'] = 2
select_mer['LABEL'] = 3
test_mer['LABEL'] = 4

In [6]:
category_lst = [buy_mer, refund_mer, compar_mer, select_mer, test_mer]
full_data = pd.concat(category_lst, ignore_index=True)
full_data.shape

(216342, 56)

In [8]:
full_data

,NUMOFBODIES,ID,PELVIS_X,PELVIS_Y,PELVIS_CONFIDENCE_LEVEL,SPINE_NAVAL_X,SPINE_NAVAL_Y,SPINE_NAVA_CONFIDENCE_LEVEL,SPINE_CHEST_X,SPINE_CHEST_Y,...,KNEE_RIGHT_CONFIDENCE_LEVEL,ANKLE_RIGHT_X,ANKLE_RIGHT_Y,ANKLE_RIGHT_CONFIDENCE_LEVEL,HEAD_X,HEAD_Y,HEAD_CONFIDENCE_LEVEL,TIMESTAMP,FRAME_NUM,LABEL
0,1,0,346.55383,640.09490,2,316.11750,576.43010,2,280.24127,522.70620,...,2,434.75787,857.02515,2,239.81610,388.57196,2,14:46:38:326,63,0
1,1,0,374.96075,649.16760,2,356.46484,581.53500,2,337.61346,522.35820,...,2,387.14087,889.73960,2,265.55145,386.76953,2,14:46:38:435,66,0
2,1,0,429.70477,641.58417,2,411.60730,574.76430,2,388.89648,514.81540,...,2,392.70930,887.81946,2,331.37190,371.98608,2,14:46:38:518,69,0
3,1,0,495.43500,632.74255,2,477.77640,565.54290,2,457.34350,506.20636,...,2,410.20886,890.77374,2,402.76740,364.13617,2,14:46:38:630,72,0
4,1,0,547.60315,616.55760,2,527.22210,550.83923,2,507.37710,493.02690,...,2,479.45310,865.62000,2,463.35953,355.15625,2,14:46:38:726,75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216337,1,0,1219.71860,662.44730,2,1230.92520,595.02515,2,1241.39250,538.59045,...,1,1134.70500,834.97217,1,1259.37120,403.96295,2,14:27:45:651,1677,4
216338,1,0,1271.88180,657.77160,2,1284.57190,588.35345,2,1297.53440,530.39655,...,2,1153.70860,863.37646,1,1314.07570,397.72302,2,14:27:45:748,1680,4
216339,1,0,1323.66500,661.49490,2,1337.05540,592.91000,2,1345.43380,533.05840,...,1,1218.74740,910.22986,1,1374.46680,396.15400,2,14:27:45:849,1683,4
216340,1,0,1368.66480,671.92664,2,1383.66700,600.05210,2,1394.43500,538.26140,...,1,1271.17430,939.52026,1,1428.83910,403.51175,2,14:27:45:945,1686,4


In [8]:
np.unique((np.array(sequence_lengths)))

array([  0, 599])

In [9]:
# 데이터와 레이블을 추출합니다.
data_without_label = full_data.drop(['LABEL', 'TIMESTAMP', 'FRAME_NUM'], axis=1).values
label_data = full_data['LABEL'].values

In [10]:
np.unique(label_data, return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int64),
 array([59900, 60499, 59900, 70682, 59900], dtype=int64))

In [11]:
np.unique(data_without_label, return_counts=True)

(array([-111.89874, -105.88568,  -94.12323, ..., 2028.04   , 2040.5305 ,
        2073.094  ]),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

In [12]:
min_sequence_length = 599

In [ ]:
sequence_start_indices = full_data.index[full_data['FRAME_NUM'] == 0].tolist()
sequence_lengths = [sequence_start_indices[i] - sequence_start_indices[i-1] for i in range(1, len(sequence_start_indices))]
sequence_lengths.insert(0, sequence_start_indices[0])

# Min sequence length
min_sequence_length = min(sequence_lengths)

In [13]:
# Adjusting the code to handle cases where the sequence length might not match the expected min_sequence_length

X_sequences = []
y_sequences = []

# Iterating over the start indices to create sequences
for i in range(len(sequence_start_indices) - 1):
    start_index = sequence_start_indices[i]
    end_index = sequence_start_indices[i + 1]
    
    # Ensure the sequence has the expected length before appending
    if end_index - start_index == min_sequence_length:
        X_sequence = full_data.drop(['LABEL', 'TIMESTAMP', 'FRAME_NUM'], axis=1).iloc[start_index:end_index].values
        y_sequence = full_data['LABEL'].iloc[start_index:end_index].values
        X_sequences.append(X_sequence)
        y_sequences.append(y_sequence[-1])  # Taking the label of the last frame for the sequence

# Converting lists to numpy arrays
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

X_sequences.shape, y_sequences.shape


((518, 599, 53), (518,))

In [14]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 1. Data Splitting
# 데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

In [15]:
print(X_train.dtype, y_train.dtype)  # 현재의 데이터 타입 확인

# # 데이터 타입 변환
# X_train = X_train.astype('float32')
# X_val = X_val.astype('float32')
# y_train = y_train.astype('int32')
# y_val = y_val.astype('int32')

float64 int64


In [16]:
import tensorflow as tf

# 2. Model Definition
# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True,  input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')  # 5 classes
])

In [17]:

# 3. Model Compilation
# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_file_path = "./model/best_lstm_model.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(model_file_path)
# - 자동 훈련 멈추기 함수 사용 : 추가 훈련 epoch 2회, 가중치 업데이트
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)


# 4. Model Training
# 모델 훈련
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000,
                    batch_size=64, callbacks=[checkpoint_cb, early_stopping_cb])

# 5. Model Evaluation
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/1000
6/6 [==============================] - 5s 549ms/step - loss: 1.6089 - accuracy: 0.2348 - val_loss: 1.6051 - val_accuracy: 0.3028
Epoch 2/1000
6/6 [==============================] - 3s 439ms/step - loss: 1.5812 - accuracy: 0.3066 - val_loss: 1.6026 - val_accuracy: 0.2477
Epoch 3/1000
6/6 [==============================] - 3s 439ms/step - loss: 1.5680 - accuracy: 0.3066 - val_loss: 1.5948 - val_accuracy: 0.2844
Epoch 4/1000
6/6 [==============================] - 2s 397ms/step - loss: 1.5553 - accuracy: 0.3315 - val_loss: 1.5880 - val_accuracy: 0.2844
Epoch 5/1000
6/6 [==============================] - 2s 389ms/step - loss: 1.5333 - accuracy: 0.3315 - val_loss: 1.5780 - val_accuracy: 0.2752
Epoch 6/1000
6/6 [==============================] - 2s 391ms/step - loss: 1.5396 - accuracy: 0.3343 - val_loss: 1.5680 - val_accuracy: 0.2844
Epoch 7/1000
6/6 [==============================] - 2s 407ms/step - loss: 1.5143 - accuracy: 0.3398 - val_loss: 1.5343 - val_accuracy: 0.3028
Epoch 

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 599, 64)           30208     
                                                                 
 dropout (Dropout)           (None, 599, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 42,789
Trainable params: 42,789
Non-trainable params: 0
_________________________________________________________________
